# Function minimization example

There are many problems where we only need to sample a single point instead of a trajectory. 
The `optimize` module is designed for this use case. It provide environments and models that help explore function landscapes in order to find points that meet a desired Min/Max condition.

## Testing a `FunctionMapper` on a benchmark function

The `FunctionMapper` is a `Swarm` with updated default parameters for solving minimization problems. It should be used with a `Function`, which is an `Environment` designed to optimize functions that return an scalar.

In this first example we will be using a benchmarking environment that represents the [Eggholder](https://en.wikipedia.org/wiki/Test_functions_for_optimization) function:

![eggholder](images/eggholder.png)

In [1]:
from fragile.optimize import FunctionMapper
from fragile.optimize.benchmarks import EggHolder

The EggHolder function is defined in the \[-512, 512\] interval.

In [2]:
print(EggHolder(), EggHolder.get_bounds())

EggHolder with function eggholder, obs shape (2,), Bounds shape float32 dtype (2,) low [-512. -512.] high [512. 512.]


And its optimum corresponds to the point (512, 404.2319) with a value of -959.64066271

In [3]:
print(EggHolder().best_state, EggHolder.benchmark)

[512.     404.2319] -959.64066271


We will be sampling the random perturbations made to the walkers from a Gaussian distribution

In [4]:
from fragile.core import NormalContinuous
def gaussian_model(env):
    # Gaussian of mean 0 and std of 10, adapted to the environment bounds
    return NormalContinuous(scale=10, loc=0., bounds=env.bounds)

To run the algorithm we need to pass the environment and the model as parameters to the `FunctionMapper`.

In [5]:
swarm = FunctionMapper(env=EggHolder,
                       model=gaussian_model,
                       n_walkers=100,
                       max_epochs=500,
                      )

In [6]:
swarm.run(report_interval=50)

HTML(value='')

## Sampling a function with a local optimizer

A simple gaussian perturbation is a very sub-optimal strategy for sampling new points. It is possible to improve the performance of the sampling process if we run a local minimization process after each random perturbation.

This can be done using the `MinimizerWrapper` class, that takes in any instance of a `Function` environment, and performs a local minimization process after each environment step.

The `MinimizerWrapper` uses `scipy.optimize.minimize` under the hood, and it can take any parameter that `scipy.optimize.minimize` supports.

In [7]:
from fragile.optimize import MinimizerWrapper
    
def optimize_eggholder():
    options = {"maxiter": 10}
    return MinimizerWrapper(EggHolder(), options=options)
    
swarm = FunctionMapper(env=optimize_eggholder,
                       model=gaussian_model,
                       n_walkers=50,
                       max_epochs=201,
                      )

In [8]:
swarm.run(report_interval=25)

HTML(value='')

This significantly increases the performance of the algorithm at the expense of using more computational resources.

## Defining a new problem using a `Function`

### The objective function

It is possible to optimize any python function that returns an scalar using a `Function`, as long as two requirements are met:

- The function needs to work with batches of points stacked across the first dimension of a numpy array. 

- It returns a vector of scalars corresponding to the values of each point evaluated.

This will allow the `Function` to vectorize the calculations on the batch of walkers.

We will also need to create a `Bounds` class that define the function domain.

In this example we will optimize a four dimensional *styblinski_tang* function, which all its coordinates defined in the \[-5, 5\] interval:
    
![styblinski_tang](images/styblinski_tang.png)

In [9]:
import numpy
from fragile.core import Bounds

def styblinski_tang(x: numpy.ndarray) -> numpy.ndarray:
    return numpy.sum(x ** 4 - 16 * x ** 2 + 5 * x, 1) / 2.0

bounds = Bounds(low=-5, high=5, shape=(4,))
print(bounds)

Bounds shape float64 dtype (4,) low [-5. -5. -5. -5.] high [5. 5. 5. 5.]


### Defining arbitrary boundary conditions

It is possible to define any kind of boundary conditions for the objective function. This can be done by passing a
callable object (such as a function) to the ``custom_domain_check`` parameter.

The ``custom_domain_check`` function has the following signature:

- It takes a batch of points as input (same as the ``function`` parameter).
- It returns an array of booleans with the same length as the input array.
- Each ``True`` value of the returned array indicates that the corresponding point is **outside** the function domain.

The ``custom_domain_check`` will only be applied to the points that are inside the defined ``Bounds``.

In [10]:
def my_custom_domain_check(x: numpy.ndarray, rewards: numpy.ndarray, n_walkers: int) -> numpy.ndarray:
    return (numpy.sum(x, axis=1) > 0.0)

To define the new environment we only need to define the appropriate ``env`` callable passing the target ``function``, the ``Bounds``, and optionally a ``custom_domain_check`` to a `Function`.

Then we can use a ``FunctionMapper`` (or any other kind of ``Swarm``) to perform the optimization process. 

In [11]:
from fragile.optimize.env import Function

In [12]:
def local_optimize_styblinsky_tang():
    function = Function(function=styblinski_tang, bounds=bounds,
                        custom_domain_check=my_custom_domain_check)
    options = {"maxiter": 10}
    return MinimizerWrapper(function, options=options)

swarm = FunctionMapper(env=local_optimize_styblinsky_tang,
                       model=lambda env: NormalContinuous(scale=1, loc=0.,
                                                          bounds=env.bounds),
                       n_walkers=50,
                       max_epochs=101)

Please be aware that if you use a ``MinimizerWrapper`` with a ``Function`` that has a ``custom_domain_check`` defined you can run into trouble.

This is because the ``scipy.optimize.minimize`` function that is running under the hood cannot account for arbitrary boundary conditions. This can lead to the ``minimize`` function returning only local minima that are
outside the defined ``custom_domain_check``.

In [13]:
swarm.run(report_interval=25)

HTML(value='')

We can see how the optimization was successful in finding the global optima of -156.66468

In [14]:
swarm.best_obs

array([-2.9035418, -2.903541 , -2.9035318, -2.903534 ], dtype=float32)

In [15]:
swarm.best_reward

-156.66466

## Optimizing a function with Evolutionary Strategies

It is possible to use the `fragile` framework to implement optimization algorithms that do not rely on a cloning process, such as Evolutionary Strategies.

If the cloning process is not needed the `NoBalance` `Swarm` is the recommended choice. It has the same features of a regular `Swarm`, but it does not perform the cloning process.

In [16]:
from fragile.core.swarm import NoBalance
from fragile.optimize.models import ESModel

In this example we will be solving a Lennard-Jonnes cluster of 4 particles, which is a 12-dimensional function with a global minima at -6.

In [17]:
from fragile.optimize.benchmarks import LennardJones

In [18]:
swarm = NoBalance(env=lambda : LennardJones(n_atoms=4),
                  model=lambda env: ESModel(bounds=env.bounds),
                  accumulate_rewards=False,
                  minimize=True,
                  n_walkers=25,
                  max_epochs=15000,
                 )

In [19]:
swarm.run(report_interval=25)

HTML(value='')